### Connecting to Big Query and Calendar

In [1]:
# using service account only to connect to BigQuery and Calendar.
from googleapiclient.discovery import build
from datetime import timedelta
from googleapiclient.http import BatchHttpRequest
from google.oauth2 import service_account
from google.cloud import bigquery
from pandas_gbq import to_gbq
from googleapiclient.errors import HttpError
import time
import datetime as datetime
import pandas as pd  
import numpy as np
from datetime import datetime, timedelta, date
import re

# from sentence_transformers import SentenceTransformer, util
# from rapidfuzz.fuzz import ratio

### Credentials

In [2]:
SERVICE_CREDENTIALS = 'service_credentials_final.json'

# Define both scopes for Calendar and BigQuery
SCOPES = [
    "https://www.googleapis.com/auth/calendar",  # Google Calendar
    "https://www.googleapis.com/auth/bigquery"   # BigQuery
]

# Create credentials with both scopes
credentials = service_account.Credentials.from_service_account_file(SERVICE_CREDENTIALS, scopes=SCOPES)

# Create BigQuery client and Calendar service using the same credentials.
client = bigquery.Client(credentials=credentials, project=credentials.project_id)
calendar_service = build('calendar', 'v3', credentials=credentials)

### Checking BigQuery Connection

In [3]:
project_id = credentials.project_id    # or just write:  'circular-maxim-436817-e3'
dataset_id = 'events_data'
# table_id_new = 'new_events'
# full_table_id = f"{project_id}.{dataset_id}.{table_id_new}"
# table_id_old = 'old_events'  # a table to store old_events (existing events).

def test_bigquery_connection(client):
    query = f"""
    SELECT COUNT(*)
    FROM `{project_id}.{dataset_id}.__TABLES_SUMMARY__`
    """
    try:
        query_job = client.query(query)
        result = next(query_job.result())  # Fetch result to ensure query runs
        print("BigQuery connection successful. Number of tables:", result[0])
    except Exception as e:
        print("Error connecting to BigQuery:", e)

# Test the BigQuery connection
test_bigquery_connection(client)

BigQuery connection successful. Number of tables: 10


### Reading tables in the dataset

In [4]:
dataset_ref = f"{project_id}.{dataset_id}"

# List tables in the dataset
tables = client.list_tables(dataset_ref)

# Print the table names
print(f"Tables in dataset {dataset_id}:")
for table in tables:
    print(table.table_id)

Tables in dataset events_data:
events_data_new_events_20241215
events_data_new_events_20241217
events_data_new_events_temp
events_data_old_events_20241213
events_data_old_events_20241215
events_data_old_events_20241217
scraped_events_20241216_123258
scraped_events_20241216_125349
scraped_events_20241216_133134
scraped_events_20241216_144020


In [5]:
today = date.today()

format_today = today.strftime("%Y%m%d")

### Load Table into pandas df

In [30]:
table_name = "events_data_new_events_temp"

# Full table reference
table_ref = f"{dataset_ref}.{table_name}"

# Load data into a DataFrame
query = f"SELECT * FROM `{table_ref}`"
combined_df = client.query(query).to_dataframe()

### Preprocessing

In [31]:
combined_df = combined_df.rename(columns={'Start_Date': 'Start Date', 'End_Date': 'End Date', 'Start_Time': 'Start Time', 'End_Time': 'End Time'})
combined_df = combined_df.drop(columns=['Unnamed: 0'])

### Creating dataset

In [118]:
# dataset_id = f"{project_id}.events_data"  # Replace with your dataset name

# # Define the dataset properties
# dataset = bigquery.Dataset(dataset_id)
# dataset.location = "EU"  # Set the dataset location (e.g., "US" or "EU")

# # Create the dataset
# dataset = client.create_dataset(dataset, timeout=30)
# print(f"Created dataset {dataset.full_dataset_id}.")

Created dataset lucky-reactor-443308-r4:events_data.


### Read and combine all .csv files

In [32]:
# import os

# output_folder = 'processed_csv'

# # Ensure the output folder exists
# os.makedirs(output_folder, exist_ok=True)

# # Columns to extract and their renamed counterparts
# columns_mapping = {
#     'Subject': 'Subject',
#     'Start_date': 'Start Date',
#     'End_date': 'End Date',
#     'Start_time': 'Start Time',
#     'End_time': 'End Time',
#     'Location': 'Location',
#     'City': 'City',
#     'Description': 'Description',
#     'Category': 'Category',
#     'Music_label': 'Music_label'
# }

# # List to store processed DataFrames
# processed_dataframes = []
# # List to store modified City column values
# city_values = []

# Function to convert German umlauts and spaces
def format_city_name(city_name):
    city_name = city_name.replace('ä', 'ae').replace('ö', 'oe').replace('ü', 'ue')
    city_name = city_name.replace('Ä', 'Ae').replace('Ö', 'Oe').replace('Ü', 'Ue')
    city_name = city_name.replace('ß', 'ss')
    city_name = city_name.replace(' ', '_')
    city_name = city_name.replace('/', '')  # Remove forward slashes
    city_name = city_name.replace('.', '').replace(',', '').replace('(', '').replace(')', '').replace('-', '')
    # List of target cities to normalize
    target_cities = ["Hamburg", "Luebeck", "Norderstedt", "Hannover", "Bremen", "Braunschweig"]
    
    # Check if city_name contains any target city
    for target_city in target_cities:
        if target_city in city_name:
            city_name = target_city
            break
    if city_name == 'inselweit':
        city_name = 'Amrum_inselweit'
    return city_name

# Convert uppercase city names to title case
combined_df['City'] = combined_df['City'].apply(lambda x: x.title() if x.isupper() else x)
# Remove rows with empty City values
combined_df = combined_df[combined_df['City'] != '_']
# Drop rows where 'City' contains 'http'
combined_df = combined_df[~combined_df['City'].str.contains('http', case=False, na=False)]
# Drop rows where 'City' is empty or null
combined_df = combined_df[combined_df['City'].notna() & (combined_df['City'].str.strip() != '')]

# # Save the combined DataFrame to a single CSV file
# combined_file_path = os.path.join(output_folder, 'combined_data.csv')
# combined_df.to_csv(combined_file_path, index=False)

# # Save the unique City names to a file
# unique_city_file_path = os.path.join(output_folder, 'unique_cities.txt')
# with open(unique_city_file_path, 'w') as f:
#     for city in sorted(set(city_values)):
#         f.write(city + '\n')

# print(f"Processed files saved to {output_folder}")
# print(f"Combined DataFrame saved as {combined_file_path}")
# print(f"Unique City names saved as {unique_city_file_path}")

In [62]:
# # Load existing dataset names from BigQuery
# def get_dataset_names():
#     datasets = client.list_datasets(project=project_id)
#     return [dataset.dataset_id for dataset in datasets]

# # Create a new dataset in BigQuery
# def create_dataset(dataset_name):
#     dataset_id = f"{project_id}.{dataset_name}"
#     dataset = bigquery.Dataset(dataset_id)
#     dataset.location = "EU"  # Set the location (update as needed)

#     try:
#         client.create_dataset(dataset, exists_ok=True)
#         print(f"Dataset '{dataset_name}' created successfully.")
#     except Exception as e:
#         print(f"Failed to create dataset '{dataset_name}': {e}")

# # Compare and create missing datasets
# def compare_and_create(city_values, dataset_names):
#     for city in city_values:
#         if city not in dataset_names:
#             print(f"Dataset for '{city}' not found. Creating...")
#             create_dataset(city)

In [63]:
# # Load city_values from processed CSV
# city_values_file = "processed_csv/unique_cities.txt"
# with open(city_values_file, "r") as f:
#     city_values = [line.strip() for line in f.readlines()]

# # Get existing dataset names from BigQuery
# dataset_names = get_dataset_names()

# # Compare and create missing datasets
# compare_and_create(city_values, dataset_names)

Dataset for '_' not found. Creating...
Dataset '_' created successfully.


### Delete dataset

In [117]:
# # Fetch all datasets in the project
# datasets = list(client.list_datasets())

# # Check if there are any datasets to delete
# if datasets:
#     print(f"Found {len(datasets)} datasets. Deleting...")
#     for dataset in datasets:
#         dataset_id = dataset.dataset_id
#         dataset_ref = client.dataset(dataset_id)
#         try:
#             # Delete the dataset, including all its tables
#             client.delete_dataset(dataset_ref, delete_contents=True, not_found_ok=True)
#             print(f"Deleted dataset: {dataset_id}")
#         except Exception as e:
#             print(f"Failed to delete dataset {dataset_id}: {e}")
# else:
#     print("No datasets found in the project.")

# print("All datasets processed.")

Found 237 datasets. Deleting...
Deleted dataset: Hitzacker_Elbe
Deleted dataset: Hodenhagen
Deleted dataset: Hoehndorf
Deleted dataset: Hoernum
Deleted dataset: Hohenaspe
Deleted dataset: Hohenlockstedt
Deleted dataset: Hohenwestedt
Deleted dataset: Hohwacht_Ostsee
Deleted dataset: Hoisdorf
Deleted dataset: Hollenstedt_NDS
Deleted dataset: Hooge
Deleted dataset: Horneburg
Deleted dataset: Hornstorf
Deleted dataset: Horst
Deleted dataset: Husum
Deleted dataset: Ihlienworth
Deleted dataset: Itzehoe
Deleted dataset: Jesteburg
Deleted dataset: Joldelund
Deleted dataset: Jork
Deleted dataset: Kalkhorst
Deleted dataset: Kaltenkirchen
Deleted dataset: Kampen_Sylt_Schleswig_Holstein
Deleted dataset: Kellenhusen
Deleted dataset: Kellinghusen
Deleted dataset: Kiebitzreihe
Deleted dataset: Kiel
Deleted dataset: Kiel_SH
Deleted dataset: Kiel_Wik
Deleted dataset: Kisdorf
Deleted dataset: Klanxbuell
Deleted dataset: Klein_OffensethSparrieshoop
Deleted dataset: Klein_Wesenberg
Deleted dataset: Kluetz

In [112]:
# dataset_id = "new_events"  # The dataset name to delete

# # Construct the full dataset ID (project ID + dataset ID)
# full_dataset_id = f"{project_id}.{dataset_id}"

# # Delete the dataset
# try:
#     client.delete_dataset(
#         full_dataset_id,
#         delete_contents=True,  # Delete all tables within the dataset
#         not_found_ok=True      # Do not raise an error if the dataset does not exist
#     )
#     print(f"Dataset '{dataset_id}' deleted successfully.")
# except Exception as e:
#     print(f"Failed to delete dataset '{dataset_id}': {e}")

Dataset 'new_events' deleted successfully.


In [52]:
# # List all datasets in the project
# datasets = client.list_datasets(project=project_id)

# # Delete each dataset
# for dataset in datasets:
#     dataset_id = dataset.dataset_id
#     full_dataset_id = f"{project_id}.{dataset_id}"
#     try:
#         client.delete_dataset(
#             full_dataset_id,
#             delete_contents=True,  # Delete all tables within the dataset
#             not_found_ok=True      # Ignore errors if the dataset doesn't exist
#         )
#         print(f"Deleted dataset: {full_dataset_id}")
#     except Exception as e:
#         print(f"Failed to delete dataset {full_dataset_id}: {e}")

Deleted dataset: lucky-reactor-443308-r4.Agathenburg
Deleted dataset: lucky-reactor-443308-r4.Ahrensburg
Deleted dataset: lucky-reactor-443308-r4.Albersdorf
Deleted dataset: lucky-reactor-443308-r4.Alkersum
Deleted dataset: lucky-reactor-443308-r4.Altenkrempe
Deleted dataset: lucky-reactor-443308-r4.Altenkrempe_OT_Kassau
Deleted dataset: lucky-reactor-443308-r4.Altenmedingen
Deleted dataset: lucky-reactor-443308-r4.Ammersbek
Deleted dataset: lucky-reactor-443308-r4.Amrum_inselweit
Deleted dataset: lucky-reactor-443308-r4.Amt_Neuhaus
Deleted dataset: lucky-reactor-443308-r4.Artlenburg
Deleted dataset: lucky-reactor-443308-r4.Aukrug
Deleted dataset: lucky-reactor-443308-r4.Aumuehle
Deleted dataset: lucky-reactor-443308-r4.Bad_Bederkesa
Deleted dataset: lucky-reactor-443308-r4.Bad_Bevensen
Deleted dataset: lucky-reactor-443308-r4.Bad_Bodenteich
Deleted dataset: lucky-reactor-443308-r4.Bad_Bramstedt
Deleted dataset: lucky-reactor-443308-r4.Bad_Fallingbostel
Deleted dataset: lucky-reactor-4

In [47]:
# # Substrings to match for deletion
# substrings_to_match = ["Hamburg", "Luebeck", "Norderstedt", "Hannover", "Bremen", "Braunschweig"]

# # Function to delete datasets
# def delete_dataset(dataset_name):
#     dataset_id = f"{project_id}.{dataset_name}"
#     try:
#         client.delete_dataset(dataset_id, delete_contents=True, not_found_ok=True)
#         print(f"Dataset '{dataset_name}' deleted successfully.")
#     except Exception as e:
#         print(f"Failed to delete dataset '{dataset_name}': {e}")

# # Get all datasets in the project
# datasets = client.list_datasets(project=project_id)

# # Iterate over datasets and delete if the name contains any substring
# for dataset in datasets:
#     dataset_name = dataset.dataset_id
#     if any(substring in dataset_name for substring in substrings_to_match):
#         print(f"Dataset '{dataset_name}' matches criteria. Deleting...")
#         delete_dataset(dataset_name)

Dataset 'Braunschweig_NDS' matches criteria. Deleting...
Dataset 'Braunschweig_NDS' deleted successfully.
Dataset 'Bremen_HB' matches criteria. Deleting...
Dataset 'Bremen_HB' deleted successfully.
Dataset 'Hamburg' matches criteria. Deleting...
Dataset 'Hamburg' deleted successfully.
Dataset 'Hannover_NDS' matches criteria. Deleting...
Dataset 'Hannover_NDS' deleted successfully.
Dataset 'Hannover_Niedersaschen' matches criteria. Deleting...
Dataset 'Hannover_Niedersaschen' deleted successfully.
Dataset 'Luebeck' matches criteria. Deleting...
Dataset 'Luebeck' deleted successfully.
Dataset 'LuebeckTravemuende' matches criteria. Deleting...
Dataset 'LuebeckTravemuende' deleted successfully.
Dataset 'Luebeck_Altstadt' matches criteria. Deleting...
Dataset 'Luebeck_Altstadt' deleted successfully.
Dataset 'Luebeck_SH' matches criteria. Deleting...
Dataset 'Luebeck_SH' deleted successfully.
Dataset 'Luebeck_St_Gertrud' matches criteria. Deleting...
Dataset 'Luebeck_St_Gertrud' deleted succ

In [34]:
# # List all datasets in the project
# datasets = client.list_datasets(project=project_id)

# # Extract dataset names
# dataset_names = [dataset.dataset_id for dataset in datasets]

In [7]:
# combined_df = pd.read_csv('processed_csv/combined_data.csv')

# combined_df.head(2)

,Subject,Start Date,End Date,Start Time,End Time,Location,City,Description,Category,Music_label
0,1000 Lights Concerts - Classical Highlights,2024-12-12,2024-12-12,17:00,,St. Nikolai-Kirchengemeinde Elmshorn,Elmshorn,https://www.biunsinnorden.de/veranstaltungen/e...,Musik,True
1,1000 Lights Concerts - Romantic Love Songs,2024-12-12,2024-12-12,20:00,,St. Nikolai-Kirchengemeinde Elmshorn,Elmshorn,https://www.biunsinnorden.de/veranstaltungen/e...,Musik,True


### Create table

In [34]:
# dataset_id = 'test_dataset'
# table_id_new = 'new_events'

# full_table_id = f"{project_id}.{dataset_id}.{table_id_new}"

# create_table_query = f"""
# CREATE OR REPLACE TABLE `{full_table_id}` (
#   `Subject` STRING,
#   `Start Date` DATE,  -- Defines Start_date as DATE type.
#   `End Date` DATE,    -- Defines Endt_date as DATE type.
#   `Start Time` STRING,
#   `End Time` STRING,
#   `Location` STRING,
#   `Description` STRING,
#   `City` STRING,
#   `Music_label` STRING
#   `Counter` INT64
# )"""
# query_job = client.query(create_table_query).result()
# print("Table created successfully.")

Table created successfully.


In [33]:
# # Truncate the table
# query = f"TRUNCATE TABLE `{full_table_id}`"

# # Execute the query
# query_job = client.query(query)  # Make an API request.
# query_job.result()  # Wait for the query to finish.

# print(f"Table {full_table_id} has been cleared.")

Table lucky-reactor-443308-r4.test_dataset.new_events has been cleared.


### Function to upload table to BigQuery

In [33]:
def uploading_table_to_big_query(df):    # Uploading data to BigQuery.
    try:
        to_gbq(df, destination_table=full_table_id, project_id=project_id, if_exists='append', credentials=credentials, progress_bar=True)
        print("Data uploaded to BigQuery successfully!")   #  append doesnt change table structure.
    except Exception as e:
        print(f"An error occurred: {e}")

# uploading_table_to_big_query(df)

# df

In [69]:
# loading_query = f"SELECT * FROM `{full_table_id}`"

# df_load = client.query(loading_query).to_dataframe()  # Execute the query and convert to DataFrame.

In [70]:
# df_load.shape

(299, 9)

In [73]:
# df_load.to_csv('Scraped_Events_Rausgegangen_HH_KI_HL_FL_new.csv')

In [72]:
# import torch  # Import PyTorch
# from transformers import AutoModel, AutoTokenizer
# from sklearn.metrics.pairwise import cosine_similarity
# from rapidfuzz.fuzz import ratio
# import pandas as pd

# # Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("deepset/gbert-base")
# model = AutoModel.from_pretrained("deepset/gbert-base")

# # Ensure the 'Subject' column is lowercase
# df['Subject'] = df['Subject'].str.lower()

# # Function to generate embeddings for the 'Subject' column
# def generate_embeddings(texts):
#     inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     # Use the [CLS] token embeddings
#     return outputs.last_hidden_state[:, 0, :].numpy()

# # Generate embeddings for the 'Subject' column
# df['event_embedding'] = list(generate_embeddings(df['Subject'].tolist()))

# # Initialize a list to store dropped rows
# dropped_rows = []

# # Prepare for duplicate removal
# to_drop = set()

# # Group by 'Start_date', 'City'
# for _, group in df.groupby(['Start_date', 'City']):
#     group = group.sort_values(by='Subject').reset_index()  # Sort within the group

#     # Compare rows within the group
#     for i in range(len(group)):
#         if group.loc[i, 'index'] in to_drop:
#             continue
#         for j in range(i + 1, len(group)):
#             if group.loc[j, 'index'] in to_drop:
#                 continue

#             # Compute cosine similarity
#             cos_similarity = cosine_similarity(
#                 [group.loc[i, 'event_embedding']], [group.loc[j, 'event_embedding']]
#             )[0][0]

#             # Compute Levenshtein similarity
#             lev_similarity = ratio(group.loc[i, 'Subject'], group.loc[j, 'Subject']) / 100.0

#             # Check if either similarity exceeds threshold
#             if cos_similarity > 0.8 or lev_similarity > 0.85:
#                 to_drop.add(group.loc[j, 'index'])  # Mark duplicate for removal
#                 dropped_rows.append(group.loc[j].to_dict())  # Add dropped row as a dictionary

# # Remove duplicates
# df = df.drop(index=to_drop).reset_index(drop=True)

# # Create a DataFrame for dropped rows
# dropped_df = pd.DataFrame(dropped_rows)

# # Drop unnecessary columns
# df = df.drop(columns=['event_embedding'])

# # Print the shape and dropped rows
# print(f"Original DataFrame Shape: {len(df) + len(dropped_df)}")
# print(f"Remaining DataFrame Rows: {len(df)}")
# print(f"No. of Dropped Rows: {len(dropped_df)}")

# # Display dropped rows for inspection
# if not dropped_df.empty:
#     print("Dropped Events:")
# else:
#     print("No duplicates were found.")

Original DataFrame Shape: 110
Remaining DataFrame Rows: 110
No. of Dropped Rows: 0
No duplicates were found.


In [ ]:
# from transformers import AutoModel, AutoTokenizer

# # Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("deepset/gbert-base")
# model = AutoModel.from_pretrained("deepset/gbert-base")

# df['Subject'] = df['Subject'].str.lower()

# # Encode the 'Subject' column to generate embeddings
# df_load['event_embedding'] = model.encode(df['Subject'].tolist()).tolist()

# # Initialize a list to store dropped rows
# dropped_rows = []

# # Prepare for duplicate removal
# to_drop = set()

# # Group by 'Start_date', 'Start_time', and 'City'
# for _, group in df_load.groupby(['Start_date', 'City']):
#     group = group.sort_values(by='Subject').reset_index()  # Sort within the group
    
#     # Compare rows within the group
#     for i in range(len(group)):
#         if group.loc[i, 'index'] in to_drop:
#             continue
#         for j in range(i + 1, len(group)):
#             if group.loc[j, 'index'] in to_drop:
#                 continue
            
#             # Compute cosine similarity
#             cos_similarity = util.cos_sim(group.at[i, 'event_embedding'], group.at[j, 'event_embedding']).item()

#             # Compute Levenshtein similarity
#             lev_similarity = ratio(group.at[i, 'Subject'], group.at[j, 'Subject']) / 100.0

#             # Check if either similarity exceeds threshold
#             if cos_similarity > 0.8 or lev_similarity > 0.85:  
#                 to_drop.add(group.loc[j, 'index'])  # Mark duplicate for removal
#                 dropped_rows.append(group.loc[j].to_dict())  # Add dropped row as a dictionary


# # Remove duplicates
# df_load = df_load.drop(index=to_drop).reset_index(drop=True)

# # Create a DataFrame for dropped rows
# dropped_df = pd.DataFrame(dropped_rows)

# # Drop unnecessary columns
# df_load = df_load.drop(columns=['event_embedding'])

# # Print the shape and dropped rows
# print(f"Original DataFrame Shape: {len(df_load) + len(dropped_df)}")
# print(f"Remaining DataFrame Rows: {len(df_load)}")
# print(f"No. of Dropped Rows: {len(dropped_df)}")

# # Display dropped rows for inspection
# print("Dropped Events:")
# print(dropped_df[['Subject', 'Start_date', 'Start_time', 'City']].head())

AttributeError: 'BertModel' object has no attribute 'encode'

In [23]:
# model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # 384 dimensional dense vector space.

# # Encode the 'Subject' column to generate embeddings
# df_load['event_embedding'] = model.encode(df['Subject'].tolist()).tolist()

# # Sort the dataframe by 'Subject', 'Start_date', and 'Start_time'
# df_load = df_load.sort_values(by=['Subject', 'Start_date', 'Start_time', 'City']).reset_index(drop=True)

# # Prepare for duplicate removal
# to_drop = set()

# # Group by 'Start_date' and 'Start_time'
# for _, group in df_load.groupby(['Start_date', 'Start_time', 'City']):
#     group = group.sort_values(by='Subject').reset_index()  # Sort within the group
    
#     # Compare rows within the group
#     for i in range(len(group)):
#         if group.loc[i, 'index'] in to_drop:
#             continue
#         for j in range(i + 1, len(group)):
#             if group.loc[j, 'index'] in to_drop:
#                 continue
            
#             # Compute cosine similarity
#             cos_similarity  = util.cos_sim(group.at[i, 'event_embedding'], group.at[j, 'event_embedding']).item()

#             # Compute Levenshtein similarity
#             lev_similarity = ratio(group.at[i, 'Subject'], group.at[j, 'Subject']) / 100.0

#             # Check if either similarity exceeds threshold
#             if cos_similarity  > 0.8 and lev_similarity > 0.9:  
#                 to_drop.add(group.loc[j, 'index'])  # Mark duplicate for removal

# # Remove duplicates
# df_load = df_load.drop(index=to_drop).reset_index(drop=True)

# # Drop unnecessary columns
# df_load = df_load.drop(columns=['event_embedding'])
# print(df_load.shape)
# df_load.head(1)

(290, 9)


,Subject,Start_date,End_date,Start_time,End_time,Location,Description,City,Music_label
0,"""Zwischen Puls & Takt"" Lesung mit Musik by HEI...",2024-12-09,2024-12-09,19:00,None,Finder‘s Haus | Raum Oslo,https://rausgegangen.de/events/zwischen-puls-t...,Lübeck,music


## Deduplication

In [34]:
deduplicated_df = combined_df.copy()

In [35]:
deduplicated_df['Counter'] = deduplicated_df.groupby(['Start Date', 'Start Time', 'Location', 'City'])['Start Date'].transform('size') - 1

In [36]:
deduplicated_df['Counter'] = deduplicated_df['Counter'].fillna(0).astype(int)

In [37]:
# Step 1: Group by the desired columns
grouped = deduplicated_df.groupby(['Start Date', 'Start Time', 'Location', 'City'])

# Step 2: Function to update the description for the first row in each group
def process_group(group):
    # If there is more than one event in the group
    if len(group) > 1:
        # Prepare the summary of dropped events
        counter = len(group) - 1
        dropped_events = "\n".join(
            f"Event {row['Subject']}: {row['Description']}" 
            for _, row in group.iloc[1:].iterrows()
        )
        # Update the description of the first event
        group.iloc[0, group.columns.get_loc('Description')] = (
            f"{counter} event(s) found at the same time/location. Please check the following:\n"
            f"{dropped_events}\n"
            f"{group.iloc[0]['Description']}"
        )
    return group.iloc[0]  # Keep only the first row in the group

# Step 3: Apply the function to each group and combine the results
result_df = grouped.apply(process_group).reset_index(drop=True)

C:\Users\heans\AppData\Local\Temp\ipykernel_16980\904577957.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_df = grouped.apply(process_group).reset_index(drop=True)


In [38]:
df_cleaned = result_df[result_df['Start Date'].astype(str).notna() & (result_df['Start Date'].astype(str).str.strip() != '')]

In [39]:
# Filter rows where Counter > 0 or Music_label is True
filtered_df = df_cleaned[(df_cleaned['Counter'] > 0) | (df_cleaned['Music_label'] == True)]
filtered_df = df_cleaned.reset_index(drop=True)

In [40]:
# Get today's date
today = pd.Timestamp(datetime.now().date())

# Calculate the date 30 days from today
thirty_days_from_today = today + timedelta(days=30)

# Convert 'Start Date' back to datetime for filtering
filtered_df['Start Date'] = pd.to_datetime(filtered_df['Start Date'], format='%Y-%m-%d')

# Filter rows where Start Date is >= today, and either Counter > 0 or filtered_label is True
filtered_df = filtered_df[
    (filtered_df['Start Date'] >= today) &
    (filtered_df['Start Date'] <= thirty_days_from_today) &  
    ((filtered_df['Counter'] > 0) | (filtered_df['Music_label'] == True))
].reset_index(drop=True)

In [41]:
filtered_df['Colour'] = 1
filtered_df.loc[filtered_df['Counter'] > 0, 'Colour'] = 4

In [42]:
filtered_df.shape

(614, 12)

In [43]:
filtered_df.Counter.value_counts()

Counter
0    551
1     58
3      3
2      2
Name: count, dtype: int64

In [44]:
filtered_df

,Subject,Start Date,Start Time,End Date,End Time,Location,City,Category,Description,Music_label,Counter,Colour
0,Drei Haselnüsse für Aschenbrödel - Der Origina...,2024-12-19,,2024-12-19,,MuK,Lübeck Altstadt,Musik,https://www.biunsinnorden.de/veranstaltungen/l...,True,0,1
1,Geigen-Duo,2024-12-19,,2024-12-19,,Spätcafé im Glockenhof,Lüneburg,Musik,https://www.biunsinnorden.de/veranstaltungen/l...,True,0,1
2,Hamburg CARD,2024-12-19,00:01,2024-12-19,,Touristinformation im Hauptbahnhof,Hamburg City,Familie / Kind,https://www.biunsinnorden.de/veranstaltungen/h...,True,0,1
3,Gärtnern in der Stadt - Die grüne Vielfalt Lüb...,2024-12-19,09:00,2024-12-19,,Museum für Natur und Umwelt,Lübeck,Ausstellungen,1 event(s) found at the same time/location. Pl...,False,1,4
4,Buddenbrooks im Behnhaus,2024-12-19,10:00,2024-12-19,,Museum Behnhaus Drägerhaus,Lübeck,Ausstellungen,1 event(s) found at the same time/location. Pl...,False,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...
609,Rev. Gregory M. Kelly & the best of Harlem Gospel,2025-01-18,20:00,2025-01-18,,Evangelisch-Lutherische St. Nikolaikirche,Elmshorn,konzerte,https://veranstaltungen.meinestadt.de/elmshorn...,True,0,1
610,Erhardt & Alexander,2025-01-18,20:00,2025-01-18,,Stadttheater Elmshorn,Elmshorn,konzerte,https://veranstaltungen.meinestadt.de/elmshorn...,True,0,1
611,"Lady Ticket 2 für 1 Groove Club"" anzeigen",2025-01-18,21:00,2025-01-18,,Studio,"Bremen, HB",music,https://www.eventbrite.de/e/lady-ticket-2-fur-...,True,0,1
612,Groove Club anzeigen,2025-01-18,21:00,2025-01-18,,studio,"Bremen, HB",music,https://www.eventbrite.de/e/groove-club-ticket...,True,0,1


### Event Comparison

* New datasets `old_events` and `new_events` are created

In [ ]:
# create_dataset('old_events')
# create_dataset('new_events')
# create_dataset('events_data')

Dataset 'events_data' created successfully.


### Creating new events table

In [45]:
table_id_new = f'{dataset_id}_new_events_{format_today}'

full_table_id = f"{dataset_ref}.{table_id_new}"

create_table_query = f"""
CREATE OR REPLACE TABLE `{full_table_id}` (
  `Subject` STRING,
  `Start Date` DATE,  -- Defines Start_date as DATE type.
  `End Date` DATE,    -- Defines Endt_date as DATE type.
  `Start Time` STRING,
  `End Time` STRING,
  `Location` STRING,
  `Description` STRING,
  `Category` STRING,
  `City` STRING,
  `Music_label` BOOL,
  `Counter` INT64,
  `Colour` INT64
)"""

query_job = client.query(create_table_query).result()
print("Table created successfully.")

Table created successfully.


In [46]:
uploading_table_to_big_query(filtered_df)  # this function uploads data to Big Query.

Data uploaded to BigQuery successfully!


In [47]:
# List tables in the dataset
tables = client.list_tables(dataset_ref)

# Retrieve table names
table_names = [table.table_id for table in tables]

# Regular expressions to match table types
new_events_pattern = r"events_data_new_events_(\d{8})"
old_events_pattern = r"events_data_old_events_(\d{8})"

# Extract and store dates for each table type
new_events = []
old_events = []

for table_name in table_names:
    new_match = re.search(new_events_pattern, table_name)
    old_match = re.search(old_events_pattern, table_name)

    if new_match:
        new_events.append((table_name, int(new_match.group(1))))  # Store table name and date
    elif old_match:
        old_events.append((table_name, int(old_match.group(1))))

# Sort the tables by date and get the latest ones
latest_new_table = max(new_events, key=lambda x: x[1], default=None)
latest_old_table = max(old_events, key=lambda x: x[1], default=None)

# Print results
if latest_new_table:
    print(f"Latest new_events table: {latest_new_table[0]} (Date: {latest_new_table[1]})")
else:
    print("No new_events table found.")

if latest_old_table:
    print(f"Latest old_events table: {latest_old_table[0]} (Date: {latest_old_table[1]})")
else:
    print("No old_events table found.")

Latest new_events table: events_data_new_events_20241219 (Date: 20241219)
Latest old_events table: events_data_old_events_20241217 (Date: 20241217)


### Copying new table to old table

In [48]:
# List tables in the dataset
tables = client.list_tables(dataset_ref)
table_names = [table.table_id for table in tables]

# Regular expression to match table names
new_events_pattern = r"events_data_new_events_(\d{8})"
old_events_pattern = r"events_data_old_events_(\d{8})"

# Extract and store dates
new_events = []
old_events = []

for table_name in table_names:
    new_match = re.search(new_events_pattern, table_name)
    old_match = re.search(old_events_pattern, table_name)
    
    if new_match:
        new_events.append((table_name, int(new_match.group(1))))
    elif old_match:
        old_events.append((table_name, int(old_match.group(1))))

# Find the latest new_events table and today's date
today = int(datetime.now().strftime("%Y%m%d"))  # Today's date in YYYYMMDD
latest_new_table = max(new_events, key=lambda x: x[1], default=None)

# Check and define the target old_events table name
if latest_new_table and latest_new_table[1] == today:
    source_table = latest_new_table[0]
    target_table = source_table.replace("new_events", "old_events")  # Replace "new" with "old"

    # SQL query to copy the table
    query = f"""
    CREATE OR REPLACE TABLE `{dataset_id}.{target_table}`
    AS
    SELECT *
    FROM `{dataset_id}.{source_table}`;
    """

    # Run the query
    query_job = client.query(query)
    query_job.result()  # Wait for query completion
    print(f"Copied {source_table} to {target_table}.")
else:
    print("No new_events table to process for today's date.")

Copied events_data_new_events_20241219 to events_data_old_events_20241219.


In [49]:
# List tables in the dataset
tables = client.list_tables(dataset_ref)
table_names = [table.table_id for table in tables]

# Regular expressions to identify table types
new_events_pattern = r"events_data_new_events_(\d{8})"
old_events_pattern = r"events_data_old_events_(\d{8})"

# Extract and store tables with their dates
new_events = []
old_events = []

for table_name in table_names:
    new_match = re.search(new_events_pattern, table_name)
    old_match = re.search(old_events_pattern, table_name)
    
    if new_match:
        new_events.append((table_name, int(new_match.group(1))))  # Table name and date
    elif old_match:
        old_events.append((table_name, int(old_match.group(1))))

# Today's date
today = int(datetime.now().strftime("%Y%m%d"))

# Find the latest new_events table (likely today's date)
latest_new_table = max(new_events, key=lambda x: x[1], default=None)

# Find the latest old_events table BEFORE today
latest_old_table = max(
    [table for table in old_events if table[1] < today],
    key=lambda x: x[1],
    default=None
)

# Load DataFrames from the tables
if latest_old_table:
    query_old = f"SELECT * FROM `{dataset_id}.{latest_old_table[0]}`"
    df_old = client.query(query_old).to_dataframe()
    print(f"Loaded df_old from table: {latest_old_table[0]}")
else:
    df_old = pd.DataFrame()  # Empty DataFrame if no table is found
    print("No valid old_events table found before today.")

if latest_new_table:
    query_new = f"SELECT * FROM `{dataset_id}.{latest_new_table[0]}`"
    df_new = client.query(query_new).to_dataframe()
    print(f"Loaded df_new from table: {latest_new_table[0]}")
else:
    df_new = pd.DataFrame()  # Empty DataFrame if no table is found
    print("No valid new_events table found for today.")

e:\venv\eventcalendar\lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Loaded df_old from table: events_data_old_events_20241217
Loaded df_new from table: events_data_new_events_20241219


In [ ]:
# table_id = "events_data_old_events_20241215"  # Replace with your table name

# # Full table reference
# table_ref = f"{project_id}.{dataset_id}.{table_id}"

# # SQL query to fetch data
# query = f"SELECT * FROM `{table_ref}`"

# # Load table into a DataFrame
# df_old = client.query(query).to_dataframe()

e:\venv\eventcalendar\lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


### Deleting table

In [37]:
# tables_to_delete = [
#     "events_data_new_events_20241221",
#     "events_data_new_events_20241220"
# ]

# # Delete each table
# for table_name in tables_to_delete:
#     table_ref = f"{project_id}.{dataset_id}.{table_name}"
#     try:
#         client.delete_table(table_ref)  # API request to delete the table
#         print(f"Deleted table: {table_ref}")
#     except Exception as e:
#         print(f"Failed to delete table {table_ref}: {e}")

Deleted table: lucky-reactor-443308-r4.events_data.events_data_new_events_20241221
Deleted table: lucky-reactor-443308-r4.events_data.events_data_new_events_20241220


### Testing

In [67]:
# Normalize time formatting
def normalize_time(value):
    if pd.notna(value) and isinstance(value, str):
        return value.zfill(5).replace('0000', '00:00')  # Ensure format HH:mm
    return value

# Apply normalization to time columns
for df in [df_old, df_new]:
    df['Start Time'] = df['Start Time'].apply(normalize_time)
    df['End Time'] = df['End Time'].apply(normalize_time)

# Process each city independently
cities = set(df_old['City'].dropna()).union(set(df_new['City'].dropna()))

# Create an empty DataFrame to store results
final_df = pd.DataFrame()

In [68]:
# Loop over each city
for city in cities:
    df_old_city = df_old[df_old['City'] == city]
    df_new_city = df_new[df_new['City'] == city]
    
    # Process old events and compare with the new dataset
    for _, old_row in df_old_city.iterrows():
        # Find matching events by Subject in the same city
        matching_subject = df_new_city[df_new_city['Subject'] == old_row['Subject']]

        if matching_subject.empty:
            # Event not found in new dataset: Mark as deleted (Red)
            deleted_event = old_row.copy()
            deleted_event['Colour'] = 11  # Red
            deleted_event['Start Time'] = normalize_time(old_row['Start Time'])
            deleted_event['End Time'] = normalize_time(old_row['End Time'])
            df_new_city = pd.concat([df_new_city, pd.DataFrame([deleted_event])], ignore_index=True)
        else:
            for idx, new_row in matching_subject.iterrows():
                # Preserve duplicate (4) if it exists in the old dataset
                if old_row['Colour'] == 4:
                    df_new_city.loc[idx, 'Colour'] = 4
                    continue

                # Check for differences in specified columns
                columns_to_check = ['Start Date', 'Start Time', 'End Date', 'End Time', 'Location', 'Category']
                differences = {}

                for col in columns_to_check:
                    if old_row[col] != new_row[col] and pd.notna(old_row[col]) and pd.notna(new_row[col]):
                        differences[col] = {
                            "Old Value": old_row[col],
                            "New Value": new_row[col]
                        }

                if differences:
                    # Event details changed: Mark as changed (Yellow)
                    df_new_city.loc[idx, 'Colour'] = 5  # Yellow
                    # Update description with original values
                    original_description = new_row.get('Description', '')
                    for key, value in differences.items():
                        original_description += f"\nOriginal {key}: {value['Old Value']}"
                    df_new_city.loc[idx, 'Description'] = original_description
                else:
                    # No changes: Set to Green (10) only if it isn't a duplicate (2)
                    if new_row.get('Colour', None) not in [4, 5, 11]:
                        df_new_city.loc[idx, 'Colour'] = 10  # Green for unchanged

    # Ensure remaining new events are marked appropriately
    for idx, new_row in df_new_city.iterrows():
        # Preserve duplicate (2) if already set
        if new_row.get('Colour', None) == 4:
            continue
        # Mark as green (10) for events without explicit updates
        if pd.isna(new_row.get('Colour', None)) or new_row['Colour'] not in [5, 11]:
            df_new_city.loc[idx, 'Colour'] = 10  # Green for unchanged

    # Append city's results to the final DataFrame
    final_df = pd.concat([final_df, df_new_city], ignore_index=True)

In [64]:
# # Create empty DataFrames to store deleted and new events
# deleted_events = pd.DataFrame()
# new_events = pd.DataFrame()

# # Loop over each city
# for city in cities:
#     # Filter datasets by city
#     df_old_city = df_old[df_old['City'] == city]
#     df_new_city = df_new[df_new['City'] == city].copy()

#     # Create a flag to track processed rows in df_new_city
#     df_new_city['Processed'] = False

#     # Process old events and compare with the new dataset
#     for _, old_row in df_old_city.iterrows():
#         # Find matching events by Subject
#         matching_subject = df_new_city[df_new_city['Subject'] == old_row['Subject']]

#         if matching_subject.empty:
#             # Event not found in new dataset: Mark as deleted (Red)
#             deleted_event = old_row.copy()
#             deleted_event['Colour'] = 11  # Red
#             deleted_event['Start Time'] = normalize_time(old_row['Start Time'])
#             deleted_event['End Time'] = normalize_time(old_row['End Time'])
#             deleted_events = pd.concat([deleted_events, pd.DataFrame([deleted_event])], ignore_index=True)
#         else:
#             # Mark matched rows in df_new_city as processed
#             for idx, new_row in matching_subject.iterrows():
#                 df_new_city.loc[idx, 'Processed'] = True

#                 # Preserve duplicate (4) if it exists in the old dataset
#                 if old_row['Colour'] == 4:
#                     df_new_city.loc[idx, 'Colour'] = 4
#                     continue

#                 # Check for differences in specified columns
#                 columns_to_check = ['Start Date', 'Start Time', 'End Date', 'End Time', 'Location', 'Category']
#                 differences = {}

#                 for col in columns_to_check:
#                     if old_row[col] != new_row[col] and pd.notna(old_row[col]) and pd.notna(new_row[col]):
#                         differences[col] = {
#                             "Old Value": old_row[col],
#                             "New Value": new_row[col]
#                         }

#                 if differences:
#                     # Event details changed: Mark as changed (Yellow)
#                     df_new_city.loc[idx, 'Colour'] = 5  # Yellow
#                     # Update description with original values
#                     original_description = new_row.get('Description', '')
#                     for key, value in differences.items():
#                         original_description += f"\nOriginal {key}: {value['Old Value']}"
#                     df_new_city.loc[idx, 'Description'] = original_description
#                 else:
#                     # No changes: Set to Green (10) only if it isn't a duplicate (2)
#                     if new_row.get('Colour', None) not in [4, 5, 11]:
#                         df_new_city.loc[idx, 'Colour'] = 10  # Green for unchanged

#     # Handle events in df_new that are not in df_old (new events)
#     for idx, new_row in df_new_city.iterrows():
#         if not new_row['Processed']:
#             # Mark as blue (7) for truly new events
#             df_new_city.loc[idx, 'Colour'] = 7

#     # Drop the temporary 'Processed' column
#     df_new_city = df_new_city.drop(columns=['Processed'])

#     # Append city's results to the final DataFrame
#     final_df = pd.concat([final_df, df_new_city], ignore_index=True)

# # Append deleted events to the final DataFrame
# final_df = pd.concat([final_df, deleted_events], ignore_index=True)


In [ ]:
# source_table = "events_data_new_events_20241215"
# target_table = "events_data_old_events_20241215"

# # SQL query to copy table
# copy_query = f"""
# CREATE OR REPLACE TABLE `{project_id}.{dataset_id}.{target_table}`
# AS
# SELECT *
# FROM `{project_id}.{dataset_id}.{source_table}`;
# """

# # Run the query
# query_job = client.query(copy_query)
# query_job.result()  # Wait for the query to finish

# print(f"Table {source_table} copied to {target_table}.")

Table events_data_new_events_20241215 copied to events_data_old_events_20241215.


In [74]:
final_df_kiel = final_df[final_df['City'] == 'Hamburg']

In [75]:
final_df_kiel

,Subject,Start Date,End Date,Start Time,End Time,Location,Description,Category,City,Music_label,Counter,Colour
264,11 Freunde,2025-01-15,2025-01-15,20:00,00:00,"Hamburg, Fabrik",https://www.livegigs.de/lesung/11-freunde/hamb...,Lesung,Hamburg,True,0,10
265,"Spurv, 44 Inch Cake",2025-01-12,2025-01-12,18:00,00:00,"Hamburg, MS Stubnitz",https://www.livegigs.de/konzert/spurv-44-inch-...,Konzert,Hamburg,True,0,10
266,Carillon-Konzert,2024-12-19,2024-12-19,12:00,00:00,Mahnmal St. Nikolai,https://veranstaltungen.meinestadt.de/hamburg/...,konzerte,Hamburg,True,0,10
267,Mittagspause,2024-12-19,2024-12-19,13:00,00:00,Kirche St. Johannis-Harvestehude,https://veranstaltungen.meinestadt.de/hamburg/...,konzerte,Hamburg,True,0,10
268,Weihnachtslieder zum mitsingen,2024-12-19,2024-12-19,18:30,00:00,Haus im Park,https://veranstaltungen.meinestadt.de/hamburg/...,konzerte,Hamburg,True,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
535,ELPHI PLAZA FüHRUNG,2024-12-18,2024-12-18,15:00,00:00,Treffpunkt: U-Bahnhof Baumwall (Elbphilharmonie),1 event(s) found at the same time/location. Pl...,Rundgänge & Fahrten,Hamburg,False,1,4
536,ALTONA - THERESIENSTADT. DIE LEBENSWEGE VON LE...,2024-12-18,2024-12-18,10:00,17:00,Altonaer Museum,2 event(s) found at the same time/location. Pl...,Ausstellungen,Hamburg,False,2,11
537,SANTA PAULI,2024-12-17,2024-12-17,16:00,23:00,Spielbudenplatz,2 event(s) found at the same time/location. Pl...,Weihnachtsmärkte,Hamburg,False,2,11
538,SAFARI TOUR AUF ST. PAULI,2024-12-17,2024-12-17,19:30,00:00,Treffpunkt: U-Bahnhof St. Pauli (Ausgang Mille...,2 event(s) found at the same time/location. Pl...,Rundgänge & Fahrten,Hamburg,False,2,4


In [76]:
final_df_kiel = final_df_kiel[final_df_kiel['Colour'] != 11]

In [ ]:
final_df_kiel

In [29]:
success_count = 0
failure_count = 0

# Loop through the DataFrame rows
for i, row in final_df_kiel.iterrows():
    # Extract row data
    start_date_str = row['Start Date']
    end_date_str = row['End Date']
    start_time_str = row['Start Time']
    end_time_str = row['End Time']
    color_id = str(row['Colour'])  # Use the Colour column for the colorId

    try:
        # Generate timestamp for event creation
        creation_timestamp = pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")

        # Append timestamp to the description
        description_with_timestamp = f"{row['Description']}\n\nEvent created on: {creation_timestamp}"

        # Check if start_time is NaN or blank (indicating an all-day event)
        if pd.isna(start_time_str) or start_time_str.strip() == "":
            # Event is all day, use only date
            start_date_obj = pd.to_datetime(start_date_str).date()  # Convert to date
            end_date_obj = pd.to_datetime(end_date_str).date() if not pd.isna(end_date_str) else start_date_obj

            # Increment end date by 1 day if start and end dates are the same
            if start_date_obj == end_date_obj:
                end_date_obj += pd.Timedelta(days=1)

            event = {
                'summary': row['Subject'],  # Use the Subject column for the event name
                'location': row['Location'],
                'start': {
                    'date': start_date_obj.isoformat(),  # Use date for all-day event
                },
                'end': {
                    'date': end_date_obj.isoformat(),  # End date for all-day event
                },
                'description': description_with_timestamp,  # Updated description
                'colorId': color_id  # Set the color based on the Colour column
            }
        else:
            # Combine date and time for start
            start_datetime_str = f"{start_date_str} {start_time_str}"  # Replace "T" with a space
            start_time_obj = pd.to_datetime(start_datetime_str)

            # Combine date and time for end
            if pd.isna(end_time_str) or end_time_str.strip() == "":
                end_time_obj = start_time_obj + pd.Timedelta(hours=1)  # Default duration: 1 hour
            else:
                end_datetime_str = f"{end_date_str} {end_time_str}"  # Replace "T" with a space
                end_time_obj = pd.to_datetime(end_datetime_str)

            # Increment end date by 1 day if start and end times overlap midnight
            if start_time_obj.date() == end_time_obj.date() and end_time_obj.time() == pd.to_datetime("00:00").time():
                end_time_obj += pd.Timedelta(days=1)

            event = {
                'summary': row['Subject'],  # Use the Subject column for the event name
                'location': row['Location'],
                'start': {
                    'dateTime': start_time_obj.strftime("%Y-%m-%dT%H:%M:%S"),
                    'timeZone': 'Europe/Berlin',  # Set to your timezone
                },
                'end': {
                    'dateTime': end_time_obj.strftime("%Y-%m-%dT%H:%M:%S"),
                    'timeZone': 'Europe/Berlin',  # Set to your timezone
                },
                'description': description_with_timestamp,  # Updated description
                'colorId': color_id  # Set the color based on the Colour column
            }

        # Create the event in Google Calendar
        created_event = calendar_service.events().insert(calendarId=calendar_id, body=event).execute()
        print(f"Event created: {created_event.get('htmlLink')}")
        success_count += 1
    except Exception as e:
        print(f"Error creating event for row {i}: {e}")
        failure_count += 1

# Print the summary of operations
print(f"Success count: {success_count}")
print(f"Failure count: {failure_count}")

Event created: https://www.google.com/calendar/event?eid=Y3FrMGF1ZDBvYzBuYXF0MHFxZm44YXNtazAgZXZlbnRjYWxlbmRhcmtpZWxAbQ
Event created: https://www.google.com/calendar/event?eid=MzIxN285a2ozZjlqNGphNXRsNmVwOWQzcWsgZXZlbnRjYWxlbmRhcmtpZWxAbQ
Event created: https://www.google.com/calendar/event?eid=aTA4MW91cjRvZHZsZGJsdWxuZWs1MDh2bGcgZXZlbnRjYWxlbmRhcmtpZWxAbQ
Event created: https://www.google.com/calendar/event?eid=bjU2a2hpaDJmaWxmdXZuMTFkaThrZ202YTAgZXZlbnRjYWxlbmRhcmtpZWxAbQ
Event created: https://www.google.com/calendar/event?eid=ZDBrNnFkNzQ1cmU0cXNlb3VpZ2sxMWpvZGcgZXZlbnRjYWxlbmRhcmtpZWxAbQ
Event created: https://www.google.com/calendar/event?eid=MGRyZWlnNWhqdGpkOXQwbzViMmlycnFha2MgZXZlbnRjYWxlbmRhcmtpZWxAbQ
Event created: https://www.google.com/calendar/event?eid=bXFwcGJjZzNrYjJqb3RkMHJoaG8xZjBmYjAgZXZlbnRjYWxlbmRhcmtpZWxAbQ
Event created: https://www.google.com/calendar/event?eid=MXBlMTIwMzJyYmc2MnJldDhzaHBpYmRvdDggZXZlbnRjYWxlbmRhcmtpZWxAbQ
Event created: https://www.google.com/ca

### Delete events in calendar

In [27]:
calendar_id = 'eventcalendarkiel@gmail.com'

# Retrieve all events
try:
    events_result = calendar_service.events().list(
        calendarId=calendar_id,
        maxResults=2500,  # Adjust if you have more events
        singleEvents=True
    ).execute()
    events = events_result.get('items', [])

    if not events:
        print("No events found.")
    else:
        # Loop through events and delete them
        for event in events:
            try:
                event_id = event['id']
                calendar_service.events().delete(calendarId=calendar_id, eventId=event_id).execute()
                print(f"Deleted event: {event.get('summary', 'Unnamed Event')}")
            except Exception as e:
                print(f"Error deleting event: {e}")

except Exception as e:
    print(f"Error retrieving events: {e}")

Deleted event: A CEREMONY OF CAROLS - ADVENTSKONZERT
Deleted event: Jam Session
Deleted event: Matthias Reim - Live 2024
Deleted event: Jalbd Shows präsentiert: Just a little bit Dangerous Fest XX
Deleted event: STABLE INVITES
Deleted event: Ell
Deleted event: WINTER VIBES PARTY
Deleted event: ELL
Deleted event: Alte Bekannte - Nix geht über LIVE! - Weihnachtsedition
Deleted event: Das Kieler Weihnachtssingen 2024
Deleted event: Live: KneipenKlassik - Winterkonzert
Deleted event: InDie Pumpe Party
Deleted event: PARTY ON 2 FLOORS
Deleted event: BREAKFASTER #6 - Beachparty
Deleted event: West Side Story
Deleted event: CRO – CRONICLES TOUR 2024
Deleted event: HE/RO - Kein Grund zur Panik Tour
Deleted event: DEICHKIND - Kids In Meinem Alter Tour 2024
Deleted event: Deichkind - Kids In Meinem Alter Tour 2024
Deleted event: Live: ARI GOLD X THEO (HipHop aus Kiel) - JAHRESABSCHLUSSKONZERT
Deleted event: Abi Wallenstein & BluesCulture feat. Steve Baker & Martin Röttger
Deleted event: FEMXLE*I

In [ ]:
# import pandas as pd
# from googleapiclient.discovery import build
# from datetime import datetime, timedelta

# # Assuming `filtered_df` is your DataFrame
# # filtered_df example structure:
# # ['Subject', 'Location', 'Start Date', 'End Date', 'Start Time', 'End Time', 'Updated_Description', 'Colour']

# success_count = 0
# failure_count = 0

# # Loop through the DataFrame rows
# for i, row in filtered_df.iterrows():
#     # Extract row data
#     start_date_str = row['Start Date']
#     end_date_str = row['End Date']
#     start_time_str = row['Start Time']
#     end_time_str = row['End Time']
#     color_id = str(row['Colour'])  # Use the Colour column for the colorId

#     try:
#         # Check if start_time is NaN (indicating an all-day event)
#         if pd.isna(start_time_str):
#             # Event is all day, use only date
#             start_date_obj = pd.to_datetime(start_date_str).date()  # Convert to date
#             end_date_obj = pd.to_datetime(end_date_str).date() if not pd.isna(end_date_str) else start_date_obj

#             event = {
#                 'summary': row['Subject'],  # Use the Subject column for the event name
#                 'location': row['Location'],
#                 'start': {
#                     'date': start_date_obj.isoformat(),  # Use date for all-day event
#                 },
#                 'end': {
#                     'date': end_date_obj.isoformat(),  # End date for all-day event
#                 },
#                 'description': row['Description'],  # Use the Description column for the description
#                 'colorId': color_id  # Set the color based on the Colour column
#             }
#         else:
#             # Combine date and time for start
#             start_datetime_str = f"{start_date_str}T{start_time_str}"
#             start_time_obj = pd.to_datetime(start_datetime_str)

#             # Combine date and time for end, or set to start if end time is missing
#             if pd.isna(end_time_str):
#                 end_time_obj = start_time_obj  # Use start time for end if missing
#             else:
#                 end_datetime_str = f"{end_date_str}T{end_time_str}"
#                 end_time_obj = pd.to_datetime(end_datetime_str)

#             event = {
#                 'summary': row['Subject'],  # Use the Subject column for the event name
#                 'location': row['Location'],
#                 'start': {
#                     'dateTime': start_time_obj.strftime("%Y-%m-%dT%H:%M:%S"),
#                     'timeZone': 'Europe/Berlin',  # Set to your timezone
#                 },
#                 'end': {
#                     'dateTime': end_time_obj.strftime("%Y-%m-%dT%H:%M:%S"),
#                     'timeZone': 'Europe/Berlin',  # Set to your timezone
#                 },
#                 'description': row['Description'],  # Use the Description column for the description
#                 'colorId': color_id  # Set the color based on the Colour column
#             }

#         # Create the event in Google Calendar
#         created_event = calendar_service.events().insert(calendarId='primary', body=event).execute()
#         print(f"Event created: {created_event.get('htmlLink')}")
#         success_count += 1
#     except Exception as e:
#         print(f"Error creating event for row {i}: {e}")
#         failure_count += 1

# # Print the summary of operations
# print(f"Success count: {success_count}")
# print(f"Failure count: {failure_count}")


C:\Users\heans\AppData\Local\Temp\ipykernel_22024\3996378728.py:43: FutureWarning: Parsed string "2024-12-13 00:00:00T " included an un-recognized timezone "T". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  start_time_obj = pd.to_datetime(start_datetime_str)


Event created: https://www.google.com/calendar/event?eid=aXNwbDVkMTJhYXA2NXFsbThlN2dtOHVmODQgdGVzdC0yMTVAbHVja3ktcmVhY3Rvci00NDMzMDgtcjQuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb20
Event created: https://www.google.com/calendar/event?eid=OW9mcjlzdGRqcGEzb2xoMDdpMjV1Zml2b28gdGVzdC0yMTVAbHVja3ktcmVhY3Rvci00NDMzMDgtcjQuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb20


C:\Users\heans\AppData\Local\Temp\ipykernel_22024\3996378728.py:43: FutureWarning: Parsed string "2024-12-13 00:00:00T00:00" included an un-recognized timezone "T". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  start_time_obj = pd.to_datetime(start_datetime_str)


Event created: https://www.google.com/calendar/event?eid=YmxlaWJxbjhtajJpNDZnZmlhcWI4MGl1MG8gdGVzdC0yMTVAbHVja3ktcmVhY3Rvci00NDMzMDgtcjQuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb20


C:\Users\heans\AppData\Local\Temp\ipykernel_22024\3996378728.py:43: FutureWarning: Parsed string "2024-12-13 00:00:00T00:01" included an un-recognized timezone "T". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  start_time_obj = pd.to_datetime(start_datetime_str)


...

Error creating event for row 1802: <HttpError 400 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "The specified time range is empty.". Details: "[{'domain': 'calendar', 'reason': 'timeRangeEmpty', 'message': 'The specified time range is empty.', 'locationType': 'parameter', 'location': 'timeMax'}]">
Error creating event for row 1803: <HttpError 400 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "The specified time range is empty.". Details: "[{'domain': 'calendar', 'reason': 'timeRangeEmpty', 'message': 'The specified time range is empty.', 'locationType': 'parameter', 'location': 'timeMax'}]">


C:\Users\heans\AppData\Local\Temp\ipykernel_22024\3996378728.py:43: FutureWarning: Parsed string "2025-01-11 00:00:00T18:30" included an un-recognized timezone "T". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  start_time_obj = pd.to_datetime(start_datetime_str)
C:\Users\heans\AppData\Local\Temp\ipykernel_22024\3996378728.py:43: FutureWarning: Parsed string "2025-01-11 00:00:00T20:00" included an un-recognized timezone "T". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  start_time_obj = pd.to_datetime(start_datetime_str)


Error creating event for row 1804: <HttpError 400 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "The specified time range is empty.". Details: "[{'domain': 'calendar', 'reason': 'timeRangeEmpty', 'message': 'The specified time range is empty.', 'locationType': 'parameter', 'location': 'timeMax'}]">
Error creating event for row 1805: <HttpError 400 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "The specified time range is empty.". Details: "[{'domain': 'calendar', 'reason': 'timeRangeEmpty', 'message': 'The specified time range is empty.', 'locationType': 'parameter', 'location': 'timeMax'}]">


C:\Users\heans\AppData\Local\Temp\ipykernel_22024\3996378728.py:43: FutureWarning: Parsed string "2025-01-11 00:00:00T23:59" included an un-recognized timezone "T". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  start_time_obj = pd.to_datetime(start_datetime_str)
C:\Users\heans\AppData\Local\Temp\ipykernel_22024\3996378728.py:43: FutureWarning: Parsed string "2025-01-12 00:00:00T16:00" included an un-recognized timezone "T". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  start_time_obj = pd.to_datetime(start_datetime_str)


Error creating event for row 1806: <HttpError 400 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "The specified time range is empty.". Details: "[{'domain': 'calendar', 'reason': 'timeRangeEmpty', 'message': 'The specified time range is empty.', 'locationType': 'parameter', 'location': 'timeMax'}]">
Success count: 676
Failure count: 1131


In [64]:
# # The big event comparison thing.
# query = f"""
# with old_events as (
#     select *,
#     rank() over (partition by Subject order by Start_date, Start_time) as Event_rank
#     from (
#         select *,
#         from {project_id}.{dataset_id}.{table_id_old}
#         where End_date >= current_date()
#         )
    
#     ),
# today_events as (
#     select *,
#     rank() over (partition by Subject order by Start_date, Start_time) as Event_rank
#     from (
#         select *,
#         from {project_id}.{dataset_id}.{table_id_old}
#         where End_date >= current_date()
#         )
#     )

#     select 
#     coalesce(t.Subject, p.Subject) AS Subject,       -- Capture events from either today or yesterday
#     coalesce(t.Location, p.Location) AS Location,    -- Use today's location if available, otherwise yesterday's
#     coalesce(t.`Start_date`, p.`Start_date`) AS Start_date,
#     coalesce(t.`End_date`, p.`End_date`) AS End_date,
#     coalesce(t.`Start_time`, p.`Start_time`) AS Start_time,
#     coalesce(t.`End_time`, p.`End_time`) AS End_time,
#     coalesce(t.Description, p.Description) AS Description,
#     coalesce(t.City, p.City) AS City,  

#     -- Description based on changes
#     CASE 
#         WHEN t.Subject IS NULL AND p.Subject IS NOT NULL THEN CONCAT(p.Description, ' | event deleted')  
#         WHEN t.Location != p.Location         THEN CONCAT(t.Description, ' | previous location: ', p.Location)
#         WHEN t.`Start_date` != p.`Start_date` AND t.`Start_time` != p.`Start_time` THEN 
#         CONCAT(t.Description, ' | previous date : ', FORMAT_DATE('%d.%m.%Y', p.`Start_date`), ' at: ', p.`Start_time`)
#         WHEN t.`Start_date` != p.`Start_date` THEN 
#         CONCAT(t.Description, ' | previous date: ', FORMAT_DATE('%d.%m.%Y', p.`Start_date`))
#         WHEN t.`Start_time` != p.`Start_time` THEN CONCAT(t.Description, ' | previous time: ', p.`Start_time`)
#         ELSE t.Description
#     END AS Updated_Description,

#     -- Status to track changes: changed, unchanged or deleted events.
#     CASE 
#         WHEN t.Subject IS NULL AND p.Subject IS NOT NULL THEN 'deleted'  -- Event present yesterday but missing today
#         WHEN t.Location != p.Location OR t.`Start_date` != p.`Start_date` OR t.`Start_time` != p.`Start_time` THEN 'changed'
#         ELSE 'unchanged'
#     END AS status


#     from today_events t
#     full outer join old_events p                        -- p stands for previous (old events).
#     on t.Subject = p.Subject and t.Event_rank = p.Event_rank and t.City = p.City
#     where coalesce(t.`End_date`, p.`End_date`) >= current_date()
# """
# # execute the query.
# query_job = client.query(query).result()

In [ ]:
# success_count = 0
# failure_count = 0

# for i, row in enumerate(query_job):

#     start_date_str = row['Start_date']
#     end_date_str = row['End_date']
#     start_time_str = row['Start_time']
#     end_time_str = row['End_time'] 

#                 # Determine the color based on the 'status' column.
#     if row['status'] == 'changed':
#         color_id = '5'   # Yellow color for changed events.
#     elif row['status'] == 'deleted':
#         color_id = '11'  # Red color for deleted events.
#     else:
#         color_id = '10'  # Green color for unchanged events.

#     # Check if start_time is NaN
#     if pd.isna(start_time_str):
#         # Event is all day, use only date
#         start_date_obj = pd.to_datetime(start_date_str).date()  # Convert to date
#         end_date_obj = pd.to_datetime(end_date_str).date()      # Convert to date

#         # Adjust for events that span midnight
#         if end_time_obj <= start_time_obj:
#             end_time_obj += timedelta(days=1)  # Move end time to the next day. [29.11.2024]

#         event = {
#             'summary': row['Subject'],
#             'location': row['Location'],
#             'start': {
#                 'date': start_date_obj.isoformat(),  # Use date for all-day event
#             },
#             'end': {
#                 'date': end_date_obj.isoformat(),  # End date for all-day event
#             },
#             'description': row['Updated_Description'],  # Include the prepared description
#             'colorId': color_id  # Set the color based on the status
#         }
#     else:
#         # Combine date and time for start
#         start_datetime_str = f"{start_date_str}T{start_time_str}"
#         start_time_obj = pd.to_datetime(start_datetime_str)

#         # Combine date and time for end, or add a default duration if end time is None
#         if pd.isna(end_time_str):  # If 'End_time' is None
#             end_datetime_str = f"{end_date_str}T{start_time_str}"
#             end_time_obj = pd.to_datetime(end_datetime_str) + timedelta(minutes=27)
#         else:
#             end_datetime_str = f"{end_date_str}T{end_time_str}"
#             end_time_obj = pd.to_datetime(end_datetime_str)

#         # Adjust for events that span midnight
#         if end_time_obj <= start_time_obj:
#             end_time_obj += timedelta(days=1)  # Move end time to the next day. [29.11.2024]

#         # if start_time_obj >= end_time_obj:  # Check for valid time range
#         #     print(f"Error: Invalid time range for row {i}. Start: {start_time_obj}, End: {end_time_obj}")
#         #     continue  # Skip this event
        
#         # Prepare the event details for Google Calendar:
#         event = {
#             'summary': row['Subject'],
#             'location': row['Location'],
#             'start': {
#                 'dateTime': start_time_obj.strftime("%Y-%m-%dT%H:%M:%S"),
#                 'timeZone': 'Europe/Berlin',  # Set to your timezone
#             },
#             'end': {
#                 'dateTime': end_time_obj.strftime("%Y-%m-%dT%H:%M:%S"),
#                 'timeZone': 'Europe/Berlin',  # Set to your timezone
#             },
#             'description': row['Updated_Description'],  # Include the prepared description
#             'colorId': color_id  # Set the color based on the status
#         }
#     try:
#         created_event = calendar_service.events().insert(calendarId='foks243dabl4@gmail.com', body=event).execute()
#         print(f"Event created: {created_event.get('htmlLink')}")
#         success_count += 1
#     except Exception as e:
#         print(f"Error creating event: {e}")
#         failure_count += 1

# print(f"success count: {success_count}")
# print(f"failure count: {failure_count}")

Event created: https://www.google.com/calendar/event?eid=OWFrNjM4bmQ3aDA2MG5ibGgxN2Z2bDFwdXMgZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=dmc3OWcybWwyaGE2MjV2NWs3ZjBjYnB1MDggZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=ZnZuMTdha2JpYXVjYnU1bHBuZ2YwOXJuMDAgZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=aDhhYzZkMTVxYWpkdDg4NmJyMHV1cWQwYnMgZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=N3Vobjh2MHIzazRpdmQ0cmI5b2hkaDhmZ2cgZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=bTBwa2o4Yjd1YTdwOXAwcjJqODVvNG1ydjggZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=NWJ1OXE2b2k1bmRtYmc2a3MybTdtcDZ0OWMgZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=aDNkdDgycjd2NDZubHVjNG90MDgyNWs4YTggZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=NDFlN3Fobm9vMXA4NXZldDhsNzg3dWdqcjAgZm9